In [3]:
""" LLM oroject By Amr El Eisawi"""
""""NVIDIA"""

'"NVIDIA'

Libary

In [4]:
import os
import shutil
import requests
import gradio as gr

from fastapi import FastAPI, UploadFile, File
import uvicorn



c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from google import genai

CONFIG

LLM API

INGESTION

In [6]:
import ollama

MODEL_NAME = "gemma3:4b" 

def ask_llm(messages):
    """
    Convert messages to prompt and ask Ollama model locally.
    """
    try:
        prompt = "\n".join(f"{msg['role']}: {msg['content']}" for msg in messages)

        # استدعاء الموديل المحلي
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}]
        )

        return response['message']['content']

    except Exception as e:
        return f"LLM Error: {str(e)}"


In [7]:
def load_documents(path: str):
    if path.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif path.endswith(".docx"):
        loader = Docx2txtLoader(path)
    else:
        raise ValueError("Unsupported file type")
    return loader.load()


def create_vectorstore(file_path: str):
    docs = load_documents(file_path)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=150
    )

    chunks = splitter.split_documents(docs)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    vs = FAISS.from_documents(chunks, embeddings)
    vs.save_local(INDEX_PATH)
    return vs


def load_vectorstore():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.load_local(
        INDEX_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )

MEMORY

In [8]:
history_store = []

def add_to_history(q, a):
    history_store.append({"q": q, "a": a})

def get_history():
    return history_store[-5:]

RAG

In [9]:
vectorstore = None

def ask_question(question: str):
    global vectorstore

    if vectorstore is None:
        return " Upload a document first.", []

    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    docs = retriever.invoke(question)

    if not docs:
        return "I couldn't find relevant information in the document.", []

    context = "\n\n".join([d.page_content for d in docs])
    citations = [f"Page {d.metadata.get('page', '?')}" for d in docs]

    history = get_history()
    history_text = "\n".join([f"Q: {h['q']} A: {h['a']}" for h in history])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a legal assistant. Use the context provided to answer questions thoroughly. "
                "Be concise but complete. If information is not in the context, say so explicitly.\n\n"
                f"Conversation History:\n{history_text}\n\n"
                f"Context:\n{context}"
            )
        },
        {"role": "user", "content": question}
    ]

    answer = ask_llm(messages)
    add_to_history(question, answer)

    return answer, citations

API

In [10]:
app = FastAPI(title="Smart Contract Assistant")

@app.post("/upload")
async def upload_contract(file: UploadFile = File(...)):
    global vectorstore

    path = f"temp_{file.filename}"
    with open(path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    try:
        vectorstore = create_vectorstore(path)
        return {"status": "indexed"}
    except Exception as e:
        return {"error": f"Failed to index document: {str(e)}"}


@app.post("/chat")
async def chat_api(q: str):
    answer, sources = ask_question(q)
    return {"answer": answer, "sources": sources}

In [11]:
API = "http://localhost:8000"

def upload_ui(file):
    files = {"file": (file.name, open(file.name, "rb"))}
    r = requests.post(f"{API}/upload", files=files)
    if r.status_code == 200:
        return r.json()
    else:
        return {"error": f"Upload failed: {r.status_code} - {r.text}"}

def chat_ui(message, history):
    r = requests.post(f"{API}/chat", params={"q": message})
    if r.status_code == 200:
        data = r.json()
        reply = data["answer"]
        sources = ", ".join(data["sources"])
        if sources:
            reply += f" Sources: {sources}"
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": reply})
        return history, history
    else:
        error_msg = f"Chat failed: {r.status_code} - {r.text}"
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": error_msg})
        return history, history


def launch_ui():
    with gr.Blocks() as demo:
        gr.Markdown(" Smart Contract Assistant")

        with gr.Tab("Upload"):
            file = gr.File()
            out = gr.JSON()
            btn = gr.Button("Upload & Index")
            btn.click(upload_ui, file, out)

        with gr.Tab("Chat"):
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            msg.submit(chat_ui, [msg, chatbot], [chatbot, chatbot])

    demo.launch()

In [12]:
if __name__ == "__main__":
    import threading

    def run_api():
        uvicorn.run(app, host="0.0.0.0", port=8000)

    threading.Thread(target=run_api).start()
    launch_ui()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
